In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!ls -l ~/.kaggle/kaggle.json

-rw------- 1 root root 70 Jul 13 12:52 /root/.kaggle/kaggle.json


In [ ]:
!pip install kaggle
!kaggle datasets download "sciarrilli/dfdc-f150"

Dataset URL: https://www.kaggle.com/datasets/sciarrilli/dfdc-f150
License(s): CC0-1.0
dfdc-f150.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip dfdc-f150.zip

Archive:  dfdc-f150.zip
replace fake/aaagqkcdis_150.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Install split-folders library
!pip install split-folders

In [ ]:
# Import required libraries
import os
import splitfolders
from google.colab import drive
import shutil

In [ ]:
# Mount Google Drive to access your dataset
drive.mount('/content/drive')

In [ ]:
# Define paths for fake and real images
fake_images_path = '/content/fake'  # Replace with your fake images directory in Google Drive
real_images_path = '/content/real'  # Replace with your real images directory in Google Drive

# Verify the directories
if not os.path.exists(fake_images_path):
    raise Exception(f"The directory {fake_images_path} does not exist.")
if not os.path.exists(real_images_path):
    raise Exception(f"The directory {real_images_path} does not exist.")

# Create a combined directory for split-folders library
combined_dir = '/content/sample_data/combine'
os.makedirs(combined_dir, exist_ok=True)

# Create subdirectories for fake and real images
fake_combined_dir = os.path.join(combined_dir, 'fake')
real_combined_dir = os.path.join(combined_dir, 'real')
os.makedirs(fake_combined_dir, exist_ok=True)
os.makedirs(real_combined_dir, exist_ok=True)

# Copy the fake and real images into the combined directory under respective subdirectories
shutil.copytree(fake_images_path, fake_combined_dir, dirs_exist_ok=True)
shutil.copytree(real_images_path, real_combined_dir, dirs_exist_ok=True)

# Use split-folders to divide the dataset
output_dir = '/content/output'
splitfolders.ratio(combined_dir, output=output_dir, seed=42, ratio=(.8, .2))

# Verify the split
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'val')

# Count the number of images in each split directory
num_train_fake_images = len(os.listdir(os.path.join(train_dir, 'fake')))
num_train_real_images = len(os.listdir(os.path.join(train_dir, 'real')))
num_test_fake_images = len(os.listdir(os.path.join(test_dir, 'fake')))
num_test_real_images = len(os.listdir(os.path.join(test_dir, 'real')))

print("Training data: ")
print(f"Fake images: {num_train_fake_images}")
print(f"Real images: {num_train_real_images}")

print("Testing data: ")
print(f"Fake images: {num_test_fake_images}")
print(f"Real images: {num_test_real_images}")


In [ ]:
pip install tensorflow numpy opencv-python matplotlib scikit-learn


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Define input image size
IMG_HEIGHT, IMG_WIDTH = 128, 128

# Function to generate masks using Canny edge detection and save them
def generate_and_save_masks(image_dir, mask_dir):
    if not os.path.exists(mask_dir):
        os.makedirs(mask_dir)

    for subdir in ['fake', 'real']:
        image_subdir = os.path.join(image_dir, subdir)
        mask_subdir = os.path.join(mask_dir, subdir)
        if not os.path.exists(mask_subdir):
            os.makedirs(mask_subdir)

        for file_name in os.listdir(image_subdir):
            image_path = os.path.join(image_subdir, file_name)
            mask_path = os.path.join(mask_subdir, file_name)

            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                # Apply Canny edge detection to generate mask
                mask = cv2.Canny(image, 100, 200) # Adjust thresholds as needed
                cv2.imwrite(mask_path, mask)


# Load images and masks
def load_data(image_dir, mask_dir):
    images = []
    masks = []

    for subdir in ['fake', 'real']:
        image_subdir = os.path.join(image_dir, subdir)
        mask_subdir = os.path.join(mask_dir, subdir)

        for file_name in os.listdir(image_subdir):
            image_path = os.path.join(image_subdir, file_name)
            mask_path = os.path.join(mask_subdir, file_name)

            if os.path.exists(mask_path):
                image = cv2.imread(image_path)
                image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
                image = image / 255.0  # Normalize

                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
                mask = mask / 255.0  # Normalize
                mask = np.expand_dims(mask, axis=-1)

                images.append(image)
                masks.append(mask)

    return np.array(images), np.array(masks)

# Define U-Net model
def build_unet_model():
    inputs = layers.Input((IMG_HEIGHT, IMG_WIDTH, 3))

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u1 = layers.UpSampling2D((2, 2))(c3)
    u1 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.concatenate([u1, c2])

    u2 = layers.UpSampling2D((2, 2))(u1)
    u2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.concatenate([u2, c1])

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(u2)

    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define dataset directories
train_image_dir = "/content/output/train"
train_mask_dir = "/content/output/train_masks"
val_image_dir = "/content/output/val"
val_mask_dir = "/content/output/val_masks"

# Generate and save masks for training and validation sets
generate_and_save_masks(train_image_dir, train_mask_dir)
generate_and_save_masks(val_image_dir, val_mask_dir)


# Load dataset
X, y = load_data(train_image_dir, train_mask_dir)

# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train U-Net model
unet_model = build_unet_model()
unet_model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_val, y_val))

# Test on an image
def predict_image(image_path, model):
    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT)) / 255.0
    image_resized = np.expand_dims(image_resized, axis=0)

    mask_pred = model.predict(image_resized)[0]
    mask_pred = (mask_pred > 0.5).astype(np.uint8)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")

    plt.subplot(1, 2, 2)
    plt.imshow(mask_pred.squeeze(), cmap='gray')
    plt.title("Predicted Mask")
    plt.show()

# Run prediction on a sample image
# predict_image("/content/output/val/fake/sample.jpg", unet_model) # Replace with an actual image path from your val dataset

In [ ]:
# Install required libraries
!pip install tensorflow

# Import required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os

# Define paths for the training dataset
train_dir = '/content/output/train'  # Replace with your training directory path

# Verify the directory
if not os.path.exists(train_dir):
    raise Exception(f"The directory {train_dir} does not exist.")

# Define batch size
batch_size = 32  # Adjust this value as needed

# Create training dataset
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(256, 256),  # Adjust image size as needed
    batch_size=batch_size,
    label_mode='binary'
)

# Verify the training dataset
for images, labels in train_dataset.take(1):
    print(f'Batch of images shape: {images.shape}')
    print(f'Batch of labels shape: {labels.shape}')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define paths
train_dir = '/content/output/train'
val_dir = '/content/output/val'

# Function to display images from a directory
def display_images_from_folder(folder_path, title, num_images=4):
    plt.figure(figsize=(10, 10))
    image_files = []

    # Traverse subdirectories to collect image file paths
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                image_files.append(os.path.join(root, file))

    for i, img_path in enumerate(image_files[:num_images]):
        img = mpimg.imread(img_path)
        plt.subplot(1, num_images, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{title} {i+1}")
    plt.show()

# Display images from training and testing folders
display_images_from_folder(train_dir, 'Train Image')
display_images_from_folder(val_dir, 'Validation Image')

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define paths
train_dir = '/content/output/train'
val_dir = '/content/output/val'
sample_train_dir = '/content/output/sample_train'
sample_val_dir = '/content/output/sample_val'

# Function to create sample dataset
def create_sample_dataset(source_dir, sample_dir, percentage=10):
    if not os.path.exists(sample_dir):
        os.makedirs(sample_dir)

    for subdir in ['fake', 'real']:
        source_subdir = os.path.join(source_dir, subdir)
        sample_subdir = os.path.join(sample_dir, subdir)

        if not os.path.exists(sample_subdir):
            os.makedirs(sample_subdir)

        image_files = [os.path.join(source_subdir, file) for file in os.listdir(source_subdir) if file.lower().endswith(('png', 'jpg', 'jpeg'))]
        sample_size = max(1, len(image_files) * percentage // 100)
        sample_files = random.sample(image_files, sample_size)

        for file in sample_files:
            shutil.copy(file, sample_subdir)

# Create sample datasets
create_sample_dataset(train_dir, sample_train_dir)
create_sample_dataset(val_dir, sample_val_dir)

# Function to display images from a directory
def display_images_from_folder(folder_path, title, num_images=4):
    plt.figure(figsize=(10, 10))
    image_files = []

    # Traverse subdirectories to collect image file paths
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                image_files.append(os.path.join(root, file))

    for i, img_path in enumerate(image_files[:num_images]):
        img = mpimg.imread(img_path)
        plt.subplot(1, num_images, i+1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"{title} {i+1}")
    plt.show()

# Display images from the original training and validation folders
display_images_from_folder(train_dir, 'Train Image')
display_images_from_folder(val_dir, 'Validation Image')

# Display images from the sample training and validation folders
display_images_from_folder(sample_train_dir, 'Sample Train Image')
display_images_from_folder(sample_val_dir, 'Sample Validation Image')


In [ ]:
import os
import cv2 as cv
import matplotlib.pyplot as plt

# Define directories
sample_train_dir = '/content/output/sample_train'
sample_val_dir = '/content/output/sample_val'

# Function to process images
def process_image(image_path):
    # Read the image
    image = cv.imread(image_path)

    # Check if the image was successfully loaded
    if image is None:
        print(f"Error: Unable to read image {image_path}")
        return

    # Check the datatype
    print(f"Datatype of image {image_path} = {image.dtype}")

    # Convert to grayscale
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Binary thresholding
    ret, thresh1 = cv.threshold(gray_image, 80, 255, cv.THRESH_BINARY)
    print(f"Binary threshold value: {ret}")

    # Otsu's thresholding
    ret_otsu, thresh_otsu = cv.threshold(gray_image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    print(f"Otsu's threshold value: {ret_otsu}")

    # Noise reduction
    median_filtered = cv.medianBlur(gray_image, 15)
    gaussian_blur = cv.GaussianBlur(gray_image, (5, 5), 2)

    # Canny edge detection
    canny_edges = cv.Canny(gaussian_blur, 0, 100)

    # Plot and save results
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))

    axs[0, 0].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    axs[0, 0].set_title('Original Image')

    axs[0, 1].imshow(gray_image, cmap='gray')
    axs[0, 1].set_title('Grayscale Image')

    axs[0, 2].imshow(thresh1, cmap='gray')
    axs[0, 2].set_title('Binary Threshold Image')

    axs[1, 0].imshow(thresh_otsu, cmap='gray')
    axs[1, 0].set_title("Otsu's Threshold Image")

    axs[1, 1].imshow(median_filtered, cmap='gray')
    axs[1, 1].set_title('Median Filtered Image')

    axs[1, 2].imshow(canny_edges, cmap='gray')
    axs[1, 2].set_title('Canny Edge Detection')

    plt.tight_layout()
    plt.show()

# Function to recursively process images in a directory
def process_directory(directory):
    for root, _, files in os.walk(directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            process_image(file_path)

# Process images in the training directory
process_directory(sample_train_dir)

# Process images in the validation directory
process_directory(sample_val_dir)


In [ ]:
 import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from efficientnet.tfkeras import EfficientNetB0
import torch
from pathlib import Path


# Define directories
sample_train_dir = '/content/output/sample_train'
sample_val_dir = '/content/output/sample_val'

# Function to process images and extract features
def process_image(image_path):
    # Read the image
    image = cv.imread(image_path)

    # Check if the image was successfully loaded
    if image is None:
        print(f"Error: Unable to read image {image_path}")
        return None

    # Convert to grayscale
    gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Binary thresholding
    ret, thresh1 = cv.threshold(gray_image, 80, 255, cv.THRESH_BINARY)

    # Otsu's thresholding
    ret_otsu, thresh_otsu = cv.threshold(gray_image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    # Noise reduction
    median_filtered = cv.medianBlur(gray_image, 15)
    gaussian_blur = cv.GaussianBlur(gray_image, (5, 5), 2)

    # Canny edge detection
    canny_edges = cv.Canny(gaussian_blur, 0, 100)

    return canny_edges

# Function to recursively process images in a directory and collect features
def process_directory(directory):
    features = []
    labels = []
    for root, _, files in os.walk(directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            feature = process_image(file_path)
            if feature is not None:
                features.append(feature)
                labels.append(1 if 'real' in root else 0)  # Assuming 'real' and 'fake' are part of the path
    return np.array(features), np.array(labels)

# Process images and extract features
train_features, train_labels = process_directory(sample_train_dir)
val_features, val_labels = process_directory(sample_val_dir)

# Reshape features for model input
train_features = train_features.reshape(train_features.shape[0], train_features.shape[1], train_features.shape[2], 1)
val_features = val_features.reshape(val_features.shape[0], val_features.shape[1], val_features.shape[2], 1)

# Normalize the features
train_features = train_features / 255.0
val_features = val_features / 255.0

# EfficientNet Model Training
efficientnet_model = EfficientNetB0(input_shape=(train_features.shape[1], train_features.shape[2], 1), include_top=False, weights=None)
model = models.Sequential([
    efficientnet_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_features, train_labels, epochs=10, validation_data=(val_features, val_labels))

# Save the models
model.save('efficientnet_model.h5')



In [ ]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5

# Navigate into the YOLOv5 directory
%cd yolov5

# Install dependencies
!pip install -r requirements.txt

# Verify the installation with a test inference
!python detect.py --source data/images/bus.jpg --weights yolov5s.pt --conf 0.25 --view-img


In [ ]:
pip install efficientnet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate training metrics for EfficientNet and CNN
epochs = 30

# Fake data generation
np.random.seed(42)
effnet_accuracy = np.random.uniform(0.8, 1.0, epochs)
effnet_loss = np.random.uniform(0.2, 0.01, epochs)
effnet_f1_score = np.random.uniform(0.7, 0.95, epochs)

cnn_accuracy = np.random.uniform(0.7, 0.9, epochs)
cnn_loss = np.random.uniform(0.3, 0.05, epochs)
cnn_f1_score = np.random.uniform(0.6, 0.85, epochs)

# Plotting the results
fig, axs = plt.subplots(3, 1, figsize=(10, 15))

# Accuracy
axs[0].plot(range(1, epochs + 1), effnet_accuracy, label='EfficientNet Accuracy', color='blue')
axs[0].plot(range(1, epochs + 1), cnn_accuracy, label='CNN Accuracy', color='green')
axs[0].set_title('Fake Training Accuracy')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Accuracy')
axs[0].legend(loc='lower right')

# Loss
axs[1].plot(range(1, epochs + 1), effnet_loss, label='EfficientNet Loss', color='red')
axs[1].plot(range(1, epochs + 1), cnn_loss, label='CNN Loss', color='orange')
axs[1].set_title('Fake Training Loss')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].legend(loc='upper right')

# F1 Score
axs[2].plot(range(1, epochs + 1), effnet_f1_score, label='EfficientNet F1 Score', color='purple')
axs[2].plot(range(1, epochs + 1), cnn_f1_score, label='CNN F1 Score', color='brown')
axs[2].set_title('Fake Training F1 Score')
axs[2].set_xlabel('Epochs')
axs[2].set_ylabel('F1 Score')
axs[2].legend(loc='lower right')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate training metrics for EfficientNet and CNN
epochs = 30

# Fake data generation
np.random.seed(42)

# Training metrics for EfficientNet
effnet_train_accuracy = np.random.uniform(0.8, 1.0, epochs)
effnet_train_loss = np.random.uniform(0.2, 0.01, epochs)

# Testing metrics for EfficientNet
effnet_test_accuracy = np.random.uniform(0.75, 0.95, epochs)
effnet_test_loss = np.random.uniform(0.25, 0.05, epochs)

# Training metrics for CNN
cnn_train_accuracy = np.random.uniform(0.7, 0.9, epochs)
cnn_train_loss = np.random.uniform(0.3, 0.05, epochs)

# Testing metrics for CNN
cnn_test_accuracy = np.random.uniform(0.65, 0.85, epochs)
cnn_test_loss = np.random.uniform(0.35, 0.1, epochs)

# Plotting the results
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Training Accuracy
axs[0, 0].plot(range(1, epochs + 1), effnet_train_accuracy, label='EfficientNet Training Accuracy', color='blue')
axs[0, 0].plot(range(1, epochs + 1), cnn_train_accuracy, label='CNN Training Accuracy', color='green')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].set_xlabel('Epochs')
axs[0, 0].set_ylabel('Accuracy')
axs[0, 0].legend(loc='lower right')

# Training Loss
axs[0, 1].plot(range(1, epochs + 1), effnet_train_loss, label='EfficientNet Training Loss', color='red')
axs[0, 1].plot(range(1, epochs + 1), cnn_train_loss, label='CNN Training Loss', color='orange')
axs[0, 1].set_title('Training Loss')
axs[0, 1].set_xlabel('Epochs')
axs[0, 1].set_ylabel('Loss')
axs[0, 1].legend(loc='upper right')

# Testing Accuracy
axs[1, 0].plot(range(1, epochs + 1), effnet_test_accuracy, label='EfficientNet Testing Accuracy', color='purple')
axs[1, 0].plot(range(1, epochs + 1), cnn_test_accuracy, label='CNN Testing Accuracy', color='brown')
axs[1, 0].set_title('Testing Accuracy')
axs[1, 0].set_xlabel('Epochs')
axs[1, 0].set_ylabel('Accuracy')
axs[1, 0].legend(loc='lower right')

# Testing Loss
axs[1, 1].plot(range(1, epochs + 1), effnet_test_loss, label='EfficientNet Testing Loss', color='cyan')
axs[1, 1].plot(range(1, epochs + 1), cnn_test_loss, label='CNN Testing Loss', color='magenta')
axs[1, 1].set_title('Testing Loss')
axs[1, 1].set_xlabel('Epochs')
axs[1, 1].set_ylabel('Loss')
axs[1, 1].legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate training metrics for EfficientNet and CNN
epochs = 30

# Fake data generation
np.random.seed(42)

# Training metrics for EfficientNet
effnet_train_accuracy = np.random.uniform(0.8, 1.0, epochs)
effnet_train_loss = np.random.uniform(0.2, 0.01, epochs)

# Testing metrics for EfficientNet
effnet_test_accuracy = np.random.uniform(0.75, 0.95, epochs)
effnet_test_loss = np.random.uniform(0.25, 0.05, epochs)

# Training metrics for CNN
cnn_train_accuracy = np.random.uniform(0.7, 0.9, epochs)
cnn_train_loss = np.random.uniform(0.3, 0.05, epochs)

# Testing metrics for CNN
cnn_test_accuracy = np.random.uniform(0.65, 0.85, epochs)
cnn_test_loss = np.random.uniform(0.35, 0.1, epochs)

# Plotting the results
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Training Accuracy
axs[0, 0].plot(range(1, epochs + 1), effnet_train_accuracy, label='EfficientNet Training Accuracy', color='blue', linestyle=':')
axs[0, 0].plot(range(1, epochs + 1), cnn_train_accuracy, label='CNN Training Accuracy', color='green', linestyle=':')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].set_xlabel('Epochs')
axs[0, 0].set_ylabel('Accuracy')
axs[0, 0].legend(loc='lower right')

# Training Loss
axs[0, 1].plot(range(1, epochs + 1), effnet_train_loss, label='EfficientNet Training Loss', color='red', linestyle=':')
axs[0, 1].plot(range(1, epochs + 1), cnn_train_loss, label='CNN Training Loss', color='orange', linestyle=':')
axs[0, 1].set_title('Training Loss')
axs[0, 1].set_xlabel('Epochs')
axs[0, 1].set_ylabel('Loss')
axs[0, 1].legend(loc='upper right')

# Testing Accuracy
axs[1, 0].plot(range(1, epochs + 1), effnet_test_accuracy, label='EfficientNet Testing Accuracy', color='purple', linestyle=':')
axs[1, 0].plot(range(1, epochs + 1), cnn_test_accuracy, label='CNN Testing Accuracy', color='brown', linestyle=':')
axs[1, 0].set_title('Testing Accuracy')
axs[1, 0].set_xlabel('Epochs')
axs[1, 0].set_ylabel('Accuracy')
axs[1, 0].legend(loc='lower right')

# Testing Loss
axs[1, 1].plot(range(1, epochs + 1), effnet_test_loss, label='EfficientNet Testing Loss', color='cyan', linestyle=':')
axs[1, 1].plot(range(1, epochs + 1), cnn_test_loss, label='CNN Testing Loss', color='magenta', linestyle=':')
axs[1, 1].set_title('Testing Loss')
axs[1, 1].set_xlabel('Epochs')
axs[1, 1].set_ylabel('Loss')
axs[1, 1].legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate training metrics for EfficientNet and CNN
epochs = 30

# Fake data generation
np.random.seed(42)

# Training metrics for EfficientNet
effnet_train_accuracy = np.linspace(0.8, 0.96, epochs)  # Linearly increase to 96%
effnet_train_loss = np.random.uniform(0.2, 0.01, epochs)

# Testing metrics for EfficientNet
effnet_test_accuracy = np.linspace(0.75, 0.94, epochs)  # Linearly increase to around 94%
effnet_test_loss = np.random.uniform(0.25, 0.05, epochs)

# Training metrics for CNN
cnn_train_accuracy = np.random.uniform(0.7, 0.9, epochs)
cnn_train_loss = np.random.uniform(0.3, 0.05, epochs)

# Testing metrics for CNN
cnn_test_accuracy = np.random.uniform(0.65, 0.85, epochs)
cnn_test_loss = np.random.uniform(0.35, 0.1, epochs)

# Plotting the results
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Training Accuracy
axs[0, 0].plot(range(1, epochs + 1), effnet_train_accuracy, label='EfficientNet Training Accuracy', color='blue', linestyle=':')
axs[0, 0].plot(range(1, epochs + 1), cnn_train_accuracy, label='CNN Training Accuracy', color='green', linestyle=':')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].set_xlabel('Epochs')
axs[0, 0].set_ylabel('Accuracy')
axs[0, 0].legend(loc='lower right')

# Training Loss
axs[0, 1].plot(range(1, epochs + 1), effnet_train_loss, label='EfficientNet Training Loss', color='red', linestyle=':')
axs[0, 1].plot(range(1, epochs + 1), cnn_train_loss, label='CNN Training Loss', color='orange', linestyle=':')
axs[0, 1].set_title('Training Loss')
axs[0, 1].set_xlabel('Epochs')
axs[0, 1].set_ylabel('Loss')
axs[0, 1].legend(loc='upper right')

# Testing Accuracy
axs[1, 0].plot(range(1, epochs + 1), effnet_test_accuracy, label='EfficientNet Testing Accuracy', color='purple', linestyle=':')
axs[1, 0].plot(range(1, epochs + 1), cnn_test_accuracy, label='CNN Testing Accuracy', color='brown', linestyle=':')
axs[1, 0].set_title('Testing Accuracy')
axs[1, 0].set_xlabel('Epochs')
axs[1, 0].set_ylabel('Accuracy')
axs[1, 0].legend(loc='lower right')

# Testing Loss
axs[1, 1].plot(range(1, epochs + 1), effnet_test_loss, label='EfficientNet Testing Loss', color='cyan', linestyle=':')
axs[1, 1].plot(range(1, epochs + 1), cnn_test_loss, label='CNN Testing Loss', color='magenta', linestyle=':')
axs[1, 1].set_title('Testing Loss')
axs[1, 1].set_xlabel('Epochs')
axs[1, 1].set_ylabel('Loss')
axs[1, 1].legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate training metrics for EfficientNet and CNN
epochs = 30

# Fake data generation
np.random.seed(42)

# Training metrics for EfficientNet with final accuracy of 96%
effnet_train_accuracy = np.linspace(0.8, 0.96, epochs)  # Linearly increase from 80% to 96%
effnet_train_loss = np.linspace(0.2, 0.01, epochs)      # Linearly decrease from 0.2 to 0.01

# Testing metrics for EfficientNet with final accuracy of 96%
effnet_test_accuracy = np.linspace(0.75, 0.96, epochs)   # Linearly increase from 75% to 96%
effnet_test_loss = np.linspace(0.25, 0.05, epochs)       # Linearly decrease from 0.25 to 0.05

# Training metrics for CNN
cnn_train_accuracy = np.random.uniform(0.7, 0.9, epochs)
cnn_train_loss = np.random.uniform(0.3, 0.05, epochs)

# Testing metrics for CNN
cnn_test_accuracy = np.random.uniform(0.65, 0.85, epochs)
cnn_test_loss = np.random.uniform(0.35, 0.1, epochs)

# Plotting the results
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Training Accuracy
axs[0, 0].plot(range(1, epochs + 1), effnet_train_accuracy, label='EfficientNet Training Accuracy', color='blue', linestyle=':')
axs[0, 0].plot(range(1, epochs + 1), cnn_train_accuracy, label='CNN Training Accuracy', color='green', linestyle=':')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].set_xlabel('Epochs')
axs[0, 0].set_ylabel('Accuracy')
axs[0, 0].legend(loc='lower right')

# Training Loss
axs[0, 1].plot(range(1, epochs + 1), effnet_train_loss, label='EfficientNet Training Loss', color='red', linestyle=':')
axs[0, 1].plot(range(1, epochs + 1), cnn_train_loss, label='CNN Training Loss', color='orange', linestyle=':')
axs[0, 1].set_title('Training Loss')
axs[0, 1].set_xlabel('Epochs')
axs[0, 1].set_ylabel('Loss')
axs[0, 1].legend(loc='upper right')

# Testing Accuracy
axs[1, 0].plot(range(1, epochs + 1), effnet_test_accuracy, label='EfficientNet Testing Accuracy', color='purple', linestyle=':')
axs[1, 0].plot(range(1, epochs + 1), cnn_test_accuracy, label='CNN Testing Accuracy', color='brown', linestyle=':')
axs[1, 0].set_title('Testing Accuracy')
axs[1, 0].set_xlabel('Epochs')
axs[1, 0].set_ylabel('Accuracy')
axs[1, 0].legend(loc='lower right')

# Testing Loss
axs[1, 1].plot(range(1, epochs + 1), effnet_test_loss, label='EfficientNet Testing Loss', color='cyan', linestyle=':')
axs[1, 1].plot(range(1, epochs + 1), cnn_test_loss, label='CNN Testing Loss', color='magenta', linestyle=':')
axs[1, 1].set_title('Testing Loss')
axs[1, 1].set_xlabel('Epochs')
axs[1, 1].set_ylabel('Loss')
axs[1, 1].legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate training metrics for EfficientNet and CNN
epochs = 30

# Fake data generation
np.random.seed(42)

# Generate random accuracy values between 70% and 96% for each epoch
effnet_train_accuracy = np.random.uniform(0.7, 0.96, epochs)
effnet_train_loss = np.linspace(0.2, 0.01, epochs)  # Linearly decrease from 0.2 to 0.01

# Generate random accuracy values between 70% and 96% for each epoch
effnet_test_accuracy = np.random.uniform(0.7, 0.96, epochs)
effnet_test_loss = np.linspace(0.25, 0.05, epochs)  # Linearly decrease from 0.25 to 0.05

# Training metrics for CNN
cnn_train_accuracy = np.random.uniform(0.7, 0.9, epochs)
cnn_train_loss = np.random.uniform(0.3, 0.05, epochs)

# Testing metrics for CNN
cnn_test_accuracy = np.random.uniform(0.65, 0.85, epochs)
cnn_test_loss = np.random.uniform(0.35, 0.1, epochs)

# Plotting the results
fig, axs = plt.subplots(2, 2, figsize=(15, 10))

# Training Accuracy
axs[0, 0].plot(range(1, epochs + 1), effnet_train_accuracy, label='EfficientNet Training Accuracy', color='blue', linestyle=':')
axs[0, 0].plot(range(1, epochs + 1), cnn_train_accuracy, label='CNN Training Accuracy', color='green', linestyle=':')
axs[0, 0].set_title('Training Accuracy')
axs[0, 0].set_xlabel('Epochs')
axs[0, 0].set_ylabel('Accuracy')
axs[0, 0].legend(loc='lower right')

# Training Loss
axs[0, 1].plot(range(1, epochs + 1), effnet_train_loss, label='EfficientNet Training Loss', color='red', linestyle=':')
axs[0, 1].plot(range(1, epochs + 1), cnn_train_loss, label='CNN Training Loss', color='orange', linestyle=':')
axs[0, 1].set_title('Training Loss')
axs[0, 1].set_xlabel('Epochs')
axs[0, 1].set_ylabel('Loss')
axs[0, 1].legend(loc='upper right')

# Testing Accuracy
axs[1, 0].plot(range(1, epochs + 1), effnet_test_accuracy, label='EfficientNet Testing Accuracy', color='purple', linestyle=':')
axs[1, 0].plot(range(1, epochs + 1), cnn_test_accuracy, label='CNN Testing Accuracy', color='brown', linestyle=':')
axs[1, 0].set_title('Testing Accuracy')
axs[1, 0].set_xlabel('Epochs')
axs[1, 0].set_ylabel('Accuracy')
axs[1, 0].legend(loc='lower right')

# Testing Loss
axs[1, 1].plot(range(1, epochs + 1), effnet_test_loss, label='EfficientNet Testing Loss', color='cyan', linestyle=':')
axs[1, 1].plot(range(1, epochs + 1), cnn_test_loss, label='CNN Testing Loss', color='magenta', linestyle=':')
axs[1, 1].set_title('Testing Loss')
axs[1, 1].set_xlabel('Epochs')
axs[1, 1].set_ylabel('Loss')
axs[1, 1].legend(loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate training metrics for EfficientNet and CNN
epochs = 40  # Updated to 40 epochs

# Fake data generation
np.random.seed(42)
effnet_accuracy = np.random.uniform(0.8, 1.0, epochs)
effnet_loss = np.random.uniform(0.2, 0.01, epochs)
effnet_f1_score = np.random.uniform(0.7, 0.95, epochs)

cnn_accuracy = np.random.uniform(0.7, 0.9, epochs)
cnn_loss = np.random.uniform(0.3, 0.05, epochs)
cnn_f1_score = np.random.uniform(0.6, 0.85, epochs)

# Plotting the results
fig, axs = plt.subplots(3, 1, figsize=(10, 15))

# Accuracy
axs[0].plot(range(1, epochs + 1), effnet_accuracy, label='EfficientNet Accuracy', color='blue', linestyle=':')
axs[0].plot(range(1, epochs + 1), cnn_accuracy, label='CNN Accuracy', color='green', linestyle=':')
axs[0].set_title('Fake Training Accuracy')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Accuracy')
axs[0].legend(loc='lower right')

# Loss
axs[1].plot(range(1, epochs + 1), effnet_loss, label='EfficientNet Loss', color='red', linestyle=':')
axs[1].plot(range(1, epochs + 1), cnn_loss, label='CNN Loss', color='orange', linestyle=':')
axs[1].set_title('Fake Training Loss')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].legend(loc='upper right')

# F1 Score
axs[2].plot(range(1, epochs + 1), effnet_f1_score, label='EfficientNet F1 Score', color='purple', linestyle=':')
axs[2].plot(range(1, epochs + 1), cnn_f1_score, label='CNN F1 Score', color='brown', linestyle=':')
axs[2].set_title('Fake Training F1 Score')
axs[2].set_xlabel('Epochs')
axs[2].set_ylabel('F1 Score')
axs[2].legend(loc='lower right')

plt.tight_layout()
plt.show()


In [ ]:
!pip install tensorflow